> `th()` is only useful for the user output when he chooses the wave

> `remove_mc()` removes the Missing Code modalities (i.e: not valid answers) into NaN

> `qual_to_quant()` because all variables are considered as categorical, transforms, if possible, a variable into a numeric one and uses the `remove_mc()` before from above. If it can't, juste return the variable cleaned

> `clean_and_stat()` computes descriptive statistics, transforms some answers to NA and removes outliers using z-score method

In [2]:
def th(digit):
    if digit == 1:
        return "1st"
    if digit == 2:
        return "2nd"
    if digit == 3:
        return "3rd"
    else:
        return f"{digit}th"

In [32]:
import numpy as np

def remove_mc(df, variable):
    
    #the Missing Codes, according to the documentation + a specific case at the end
    to_remove = ["Don't know",
                 "Refusal",
                 "Implausible value/suspected wrong",
                 "Implausible/suspected wrong",
                 "Not codable",
                 "Not answered",
                 "Not yet coded",
                 "Not applicable",
                 "Not applicable (not yet part of sample)",
                 "Not applicable (no participation in this wave)",
                 "Not applicable (respondent was not listed as household member in this wave)",
                 "Not applicable (respondent has an End-of-Life interview in previous waves)",
                 "Not applicable (no main interview done)",
                 "Not applicable (other reason)",
                 "Not applicable (missing by design)",
                 "Still living at residence"]
    
    return df[variable].replace(to_remove, np.NaN)

In [31]:
import pandas as pd
import numpy as np
import streamlit as st

def qual_to_quant(df, variable):
    
    #remove the missing codes from above
    df[variable] = remove_mc(df, variable)
    
    #if possible, convert a variable to a numeric one
    try: 
        df[variable] = df[variable].astype("float")
    except ValueError:
        pass
    
    return df[variable]

In [33]:
import pandas as pd
import numpy as np
import streamlit as st
from scipy import stats


def clean_and_stat(df, variable, z_threshold=3, full_na=False):
    
    #if var is qualitative, print percentage of each modality
    if df[variable].dtype == "object":
        counts = df[variable].value_counts()
        n = sum(counts)
        for i in range(len(counts)):
            counts[i] = str(round(counts[i]/n *100, 2))+'%'
        st.write(counts)  
    
    #if var is quantitative, print mean, sd and median + remove outliers + 
    else:
        
        #according to the documentation, financial amount that
        #have been answered "Refusal" and "Don't know" are code as follow:
        #-9999991 and -9999992. So we decide to transform them to nan
        threshold = -9999990
        to_change_to_nan = df[variable]<threshold 
        df[variable] = df[variable].where(~to_change_to_nan, np.nan)
        
        #remove outliers
        if full_na == True:
            pass
        else:
            mean = df[variable].mean()
            std = df[variable].std()
            z_scores = (df[variable] - mean) / std
            df.loc[z_scores > z_threshold, variable] = np.nan
        
        #compute descriptive stats and print them
        mean = round(df[variable].mean(),2)
        sd = round(df[variable].std(),2)
        median = round(df[variable].median(),2)
        st.write(f"Mean: {mean}")
        st.write(f"Std: {sd}")
        st.write(f"Median: {median}")